In [31]:
using DataFrames

# ============================================================
# 1. Flow Matrix f_ij  (from ICA 10)
# ============================================================
f = [
    0   5  10  15  20;
    0   0   2  20  10;
    0   0   0   2   5;
    0   0   0   0  15;
    0   0   0   0   0
]

# ============================================================
# 2. Correct Coordinates for "1 2 3 / 4 5"
#     (offset bottom row: 4 under 2, 5 under 3)
# ============================================================
positions = Dict(
    1 => (1, 1),
    2 => (2, 1),
    3 => (3, 1),
    4 => (2, 2),   # directly below dept. 2
    5 => (3, 2)    # directly below dept. 3
)

# ============================================================
# 3. Helper Functions
# ============================================================
function distance_matrix(layout)
    n = length(layout)
    d = zeros(Int, n, n)
    for i in 1:n, j in 1:n
        if i != j
            xi, yi = positions[layout[i]]
            xj, yj = positions[layout[j]]
            d[i, j] = abs(xi - xj) + abs(yi - yj)
        end
    end
    return d
end

function compute_Z(f::Matrix{Int}, d::Matrix{Int})
    total = 0
    for i in 1:size(f,1), j in 1:size(f,2)
        total += f[i,j] * d[i,j]
    end
    return total
end

function iterate_once(layout::Vector{Int}, f::Matrix{Int})
    n = length(layout)
    baseZ = compute_Z(f, distance_matrix(layout))
    rows = NamedTuple[]
    for i in 1:n-1, j in i+1:n
        new_layout = copy(layout)
        new_layout[i], new_layout[j] = new_layout[j], new_layout[i]
        z = compute_Z(f, distance_matrix(new_layout))
        push!(rows, (Exchange = "$(layout[i])-$(layout[j])",
                     NewLayout = join(new_layout, " "),
                     Z_new = z,
                     ΔZ = baseZ - z))
    end
    df = DataFrame(rows)
    minZ = minimum(df.Z_new)
    best = df[df.Z_new .== minZ, :]
    return (df, baseZ, minZ, best)
end

# ============================================================
# 4. Iterative Search Until No Improvement
# ============================================================
function pairwise_exchange_full(f::Matrix{Int}, initial_layout::Vector{Int})
    layouts = [initial_layout]
    Z_current = compute_Z(f, distance_matrix(initial_layout))
    iter = 1

    println("====================================================")
    println(" ICA 10 — Pairwise Exchange Algorithm (Offset Layout)")
    println("====================================================")
    println("\nInitial Layout: ", join(initial_layout, " "), "   Z₀ = ", Z_current, "\n")

    while true
        println("----------------------------------------------------")
        println("Iteration ", iter, ": Current Layout(s)")
        for l in layouts
            println(" → ", join(l, " "))
        end
        println("----------------------------------------------------")

        all_results = DataFrame(Exchange=String[], NewLayout=String[], Z_new=Int[], ΔZ=Int[])
        new_candidates = Vector{Vector{Int}}()

        for l in layouts
            (df, Z_old, Z_min, best) = iterate_once(l, f)
            println("\n>>> From Layout ", join(l, " "), ":  Zₖ = ", Z_old)
            show(df, allcols=true, allrows=true)
            println("\nBest from this layout:")
            display(best)
            append!(all_results, df)
            for b in best.NewLayout
                push!(new_candidates, parse.(Int, split(b)))
            end
        end

        minZ_iter = minimum(all_results.Z_new)
        best_global = all_results[all_results.Z_new .== minZ_iter, :]

        println("\n====================================================")
        println(" Iteration ", iter, " Summary")
        println("====================================================")
        println("Minimum Z_iter = ", minZ_iter)
        println("Best Global Layout(s):")
        display(best_global)

        if minZ_iter < Z_current
            Z_current = minZ_iter
            layouts = [parse.(Int, split(l)) for l in best_global.NewLayout]
            println("\n↓ Improvement Found — Proceeding to Next Iteration ↓\n")
            iter += 1
        else
            println("\n✅ No Further Improvement — Algorithm Converged ✅")
            println("\nFinal Optimal Layout(s):")
            for l in layouts
                println(" → ", join(l, " "))
            end
            println("Final Optimal Z* = ", Z_current)
            break
        end
    end
end

# ============================================================
# 5. Run Algorithm
# ============================================================
initial_layout = [1, 2, 3, 4, 5]  # matches "1 2 3 / 4 5" (offset geometry)
pairwise_exchange_full(f, initial_layout)


 ICA 10 — Pairwise Exchange Algorithm (Offset Layout)

Initial Layout: 1 2 3 4 5   Z₀ = 181

----------------------------------------------------
Iteration 1: Current Layout(s)
 → 1 2 3 4 5
----------------------------------------------------

>>> From Layout 1 2 3 4 5:  Zₖ = 181
10×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 1-2       2 1 3 4 5    168     13
   2 │ 1-3       3 2 1 4 5    151     30
   3 │ 1-4       4 2 3 1 5    171     10
   4 │ 1-5       5 2 3 4 1    171     10
   5 │ 2-3       1 3 2 4 5    189     -8
   6 │ 2-4       1 4 3 2 5    176      5
   7 │ 2-5       1 5 3 4 2    151     30
   8 │ 3-4       1 2 4 3 5    181      0
   9 │ 3-5       1 2 5 4 3    176      5
  10 │ 4-5       1 2 3 5 4    189     -8
Best from this layout:


2×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 1-3       3 2 1 4 5    151     30
   2 │ 2-5       1 5 3 4 2    151     30


 Iteration 1 Summary
Minimum Z_iter = 151
Best Global Layout(s):


2×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 1-3       3 2 1 4 5    151     30
   2 │ 2-5       1 5 3 4 2    151     30


↓ Improvement Found — Proceeding to Next Iteration ↓

----------------------------------------------------
Iteration 2: Current Layout(s)
 → 3 2 1 4 5
 → 1 5 3 4 2
----------------------------------------------------

>>> From Layout 3 2 1 4 5:  Zₖ = 151
10×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 3-2       2 3 1 4 5    158     -7
   2 │ 3-1       1 2 3 4 5    181    -30
   3 │ 3-4       4 2 1 3 5    151      0
   4 │ 3-5       5 2 1 4 3    151      0
   5 │ 2-1       3 1 2 4 5    169    -18
   6 │ 2-4       3 4 1 2 5    146      5
   7 │ 2-5       3 5 1 4 2    145      6
   8 │ 1-4       3 2 4 1 5    171    -20
   9 │ 1-5       3 2 5 4 1    166    -15
  10 │ 4-5       3 2 1 5 4    163    -12
Best from this layout:


...


>>> From Layout 2 5 1 4 3:  Zₖ = 135
10×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 2-5       5 2 1 4 3    151    -16
   2 │ 2-1       1 5 2 4 3    166    -31
   3 │ 2-4       4 5 1 2 3    163    -28
   4 │ 2-3       3 5 1 4 2    145    -10
   5 │ 5-1       2 1 5 4 3    163    -28
   6 │ 5-4       2 4 1 5 3    140     -5
   7 │ 5-3       2 3 1 4 5    158    -23
   8 │ 1-4       2 5 4 1 3    171    -36
   9 │ 1-3       2 5 3 4 1    151    -16
  10 │ 4-3       2 5 1 3 4    135      0
Best from this layout:


1×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 4-3       2 5 1 3 4    135      0


 Iteration 4 Summary
Minimum Z_iter = 135
Best Global Layout(s):


2×4 DataFrame
 Row │ Exchange  NewLayout  Z_new  ΔZ    
     │ String    String     Int64  Int64 
─────┼───────────────────────────────────
   1 │ 4-3       2 5 1 3 4    135      0
   2 │ 4-3       2 5 1 3 4    135      0


✅ No Further Improvement — Algorithm Converged ✅

Final Optimal Layout(s):
 → 2 5 1 4 3
 → 2 5 1 4 3
Final Optimal Z* = 135
